# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_df = pd.merge(student_data, school_data, how="left", on=["school_name"])

#creating boolean columns for passing
school_df = school_df.reindex(columns=["Student ID", "student_name", "gender", "grade", "school_name", "reading_score", "passing_math", "math_score", "passing_reading", "passing_overall", "School ID", "type", "size", "budget"])
def passing(read_column, write_column):
    school_df[write_column] = school_df[read_column].apply(lambda x: True if x >= 70 else False)
passing("math_score", "passing_math")
passing("reading_score", "passing_reading")
school_df["passing_overall"] = ((school_df["math_score"] + school_df["reading_score"])/2).apply(lambda x: True if x >= 70 else False)

print(school_df.dtypes)
# school_df.count()
school_df.head()

Student ID          int64
student_name       object
gender             object
grade              object
school_name        object
reading_score       int64
passing_math         bool
math_score          int64
passing_reading      bool
passing_overall      bool
School ID           int64
type               object
size                int64
budget              int64
dtype: object


,Student ID,student_name,gender,grade,school_name,reading_score,passing_math,math_score,passing_reading,passing_overall,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,True,79,False,True,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,False,61,True,True,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,False,60,True,True,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,False,58,False,False,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,True,84,True,True,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [21]:
# number of schools & Students
schools_n = len(school_df["school_name"].unique())
schools_n
students_n = len(school_df)

# total budget
total_budget = school_data["budget"].sum()

# average math & reading scores
math_ave = round(school_df["math_score"].mean(), 2)
reading_ave = round(school_df["reading_score"].mean(), 2)

# % passing math and reading
# passing = round((len(school_df[school_df["math_score"] >= 70]) + len(school_df[school_df["reading_score"] >= 70])) / 2 / len(school_df) * 100, 2)
# passing_math = round(len(school_df[school_df["math_score"] >= 70]) / len(school_df) * 100, 2)
# passing_reading = round(len(school_df[school_df["reading_score"] >= 70]) / len(school_df) * 100, 2)

# Creaing dictionary for dataframe
district_dict = {"Total Schools": [schools_n], "Total Students": [students_n], "Total Budget": "$" + str(total_budget), 
     "Average Math Score": math_ave, "Average Reading Score": reading_ave, 
     "% Passing Math": 0, "% Passing Reading": 0, "Overall Passing Rate": 0}

def passPerc(column):
    percent = round(len(school_df[school_df[column] == True]) / len(school_df[column]) * 100, 2)
    district_dict[column] = percent

passPerc("passing_overall")
passPerc("passing_math")
passPerc("passing_reading")

district_dict

{'Total Schools': [15],
 'Total Students': [39170],
 'Total Budget': '$24649428',
 'Average Math Score': 78.99,
 'Average Reading Score': 81.88,
 '% Passing Math': 0,
 '% Passing Reading': 0,
 'Overall Passing Rate': 0,
 'passing_overall': 89.39,
 'passing_math': 74.98,
 'passing_reading': 85.81}

In [22]:
district_summary = pd.DataFrame(district_dict)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,passing_overall,passing_math,passing_reading
0,15,39170,$24649428,78.99,81.88,0,0,0,89.39,74.98,85.81


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [26]:
# Creating df with mean scores
school_group = school_df.groupby(["school_name"])
school_group_reduced = school_group[["school_name", "reading_score", "math_score", "passing_math", "passing_reading", "passing_overall"]].mean()
school_summary = pd.merge(school_group_reduced, school_data, how="left", on=["school_name"])
school_summary = school_summary.reindex(columns=["school_name", "type", "size", "budget", "Per Student Budget", "math_score", "reading_score", "passing_math", "passing_reading", "passing_overall"])


# Budget per student
school_summary["Per Student Budget"] = school_summary["budget"] / school_summary["size"]

# Passing percentages
def passPerc_school(column):
    percent = school_summary[column] * 100
    school_summary[column] = percent

passPerc_school("passing_overall")
passPerc_school("passing_math")
passPerc_school("passing_reading")

# Final formatting
school_summary = round(school_summary, 2)
school_summary = school_summary.rename(columns={"school_name": "School", "type": "Type", "size": "Total Students", "budget": "Budget", "math_score": "Math Score", "reading_score": "Reading Score",  "passing_reading": "% Passing Reading", "passing_overall": "Overall Passing Rate", "passing_math": "% Passing Math"})

school_summary

,School,Type,Total Students,Budget,Per Student Budget,Math Score,Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,85.19
1,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,99.57
2,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,84.67
3,Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,84.78
4,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,99.46
5,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,84.88
6,Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,98.59
7,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,84.98
8,Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,84.98
9,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,99.17


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [27]:
school_summary.sort_values(["Overall Passing Rate"], ascending=False)

,School,Type,Total Students,Budget,Per Student Budget,Math Score,Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
1,Cabrera High School,Charter,1858,1081356,582.0,83.06,83.98,94.13,97.04,99.57
4,Griffin High School,Charter,1468,917500,625.0,83.35,83.82,93.39,97.14,99.46
11,Shelton High School,Charter,1761,1056600,600.0,83.36,83.73,93.87,95.85,99.38
13,Wilson High School,Charter,2283,1319574,578.0,83.27,83.99,93.87,96.54,99.26
14,Wright High School,Charter,1800,1049400,583.0,83.68,83.96,93.33,96.61,99.22
9,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,99.17
12,Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,99.08
6,Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,98.59
0,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,85.19
7,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,84.98


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [28]:
school_summary.sort_values(["Overall Passing Rate"])

,School,Type,Total Students,Budget,Per Student Budget,Math Score,Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
2,Figueroa High School,District,2949,1884411,639.0,76.71,81.16,65.99,80.74,84.67
10,Rodriguez High School,District,3999,2547363,637.0,76.84,80.74,66.37,80.22,84.75
3,Ford High School,District,2739,1763916,644.0,77.10,80.75,68.31,79.30,84.78
5,Hernandez High School,District,4635,3022020,652.0,77.29,80.93,66.75,80.86,84.88
7,Huang High School,District,2917,1910635,655.0,76.63,81.18,65.68,81.32,84.98
8,Johnson High School,District,4761,3094650,650.0,77.07,80.97,66.06,81.22,84.98
0,Bailey High School,District,4976,3124928,628.0,77.05,81.03,66.68,81.93,85.19
6,Holden High School,Charter,427,248087,581.0,83.80,83.81,92.51,96.25,98.59
12,Thomas High School,Charter,1635,1043130,638.0,83.42,83.85,93.27,97.31,99.08
9,Pena High School,Charter,962,585858,609.0,83.84,84.04,94.59,95.95,99.17


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [29]:
# grade_math = school_summary["School"]
grade_math = {}
grades = ['9th', '10th', '11th', '12th']

# grade = school_df[["school_name", "math_score"]][school_df["grade"] == "9th"].groupby(["school_name"]).mean()

def gradeAve(input_column, output_df):
    for item in grades:
        grade_mean = school_df[["school_name", input_column]][school_df["grade"] == item].groupby(["school_name"]).mean()
#         grade_mean = pd.DataFrame(grade_mean)
#         print(grade_mean[input_column])
        output_df[item] = grade_mean[input_column]
       
# Couldn't find a way to export this to a df properly. I tried every way I could think to export it dictionary key pairs,
# but it always came out as a group object that couldn't be added to a dataframe the way I wanted
gradeAve("math_score", grade_math)     
# # grade_math = .mean()
grade_math


{'9th': school_name
 Bailey High School       77.083676
 Cabrera High School      83.094697
 Figueroa High School     76.403037
 Ford High School         77.361345
 Griffin High School      82.044010
 Hernandez High School    77.438495
 Holden High School       83.787402
 Huang High School        77.027251
 Johnson High School      77.187857
 Pena High School         83.625455
 Rodriguez High School    76.859966
 Shelton High School      83.420755
 Thomas High School       83.590022
 Wilson High School       83.085578
 Wright High School       83.264706
 Name: math_score, dtype: float64, '10th': school_name
 Bailey High School       76.996772
 Cabrera High School      83.154506
 Figueroa High School     76.539974
 Ford High School         77.672316
 Griffin High School      84.229064
 Hernandez High School    77.337408
 Holden High School       83.429825
 Huang High School        75.908735
 Johnson High School      76.691117
 Pena High School         83.372000
 Rodriguez High School   

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [30]:
grade_reading = {}
grades = ['9th', '10th', '11th', '12th']

# grade = school_df[["school_name", "math_score"]][school_df["grade"] == "9th"].groupby(["school_name"]).mean()

def gradeAve(input_column, output_df):
    for item in grades:
        grade_mean = school_df[["school_name", input_column]][school_df["grade"] == item].groupby(["school_name"]).mean()
#         grade_mean = pd.DataFrame(grade_mean)
#         print(grade_mean[input_column])
        output_df[item] = grade_mean[input_column]
       
# Used the same function here, with the same result.
gradeAve("reading_score", grade_reading)     
# # grade_math = .mean()
grade_reading


{'9th': school_name
 Bailey High School       81.303155
 Cabrera High School      83.676136
 Figueroa High School     81.198598
 Ford High School         80.632653
 Griffin High School      83.369193
 Hernandez High School    80.866860
 Holden High School       83.677165
 Huang High School        81.290284
 Johnson High School      81.260714
 Pena High School         83.807273
 Rodriguez High School    80.993127
 Shelton High School      84.122642
 Thomas High School       83.728850
 Wilson High School       83.939778
 Wright High School       83.833333
 Name: reading_score, dtype: float64, '10th': school_name
 Bailey High School       80.907183
 Cabrera High School      84.253219
 Figueroa High School     81.408912
 Ford High School         81.262712
 Griffin High School      83.706897
 Hernandez High School    80.660147
 Holden High School       83.324561
 Huang High School        81.512386
 Johnson High School      80.773431
 Pena High School         83.612000
 Rodriguez High School

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [31]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [32]:
school_bins_budget = school_summary
school_bins_budget["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names)
school_bins_budget = school_bins_budget.reindex(columns=["Spending Ranges (Per Student)", "Math Score", "Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"])
school_bins_budget.sort_values(["Spending Ranges (Per Student)"])
school_bins_budget = school_bins_budget.groupby("Spending Ranges (Per Student)")
school_bins_budget.mean()

,Math Score,Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.452500,83.935000,93.460000,96.610000,99.160000
$585-615,83.600000,83.885000,94.230000,95.900000,99.275000
$615-645,79.078333,81.891667,75.668333,86.106667,89.655000
$645-675,76.996667,81.026667,66.163333,81.133333,84.946667


## Scores by School Size

* Perform the same operations as above, based on school size.

In [46]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [54]:
school_bins_size = school_summary
school_bins_size["School Size"] = pd.cut(school_summary["Total Students"], size_bins, labels=group_names)
school_bins_size = school_bins_size.reindex(columns=["School Size", "Math Score", "Reading Score", "% Passing Math", "% Passing Reading", "Overall Passing Rate"])
school_bins_size = school_bins_size.groupby("School Size").mean()
school_bins_size

,Math Score,Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.820,83.92500,93.55000,96.10000,98.88000
Medium (1000-2000),83.374,83.86800,93.59800,96.79000,99.34200
Large (2000-5000),77.745,81.34375,69.96375,82.76625,86.68625


## Scores by School Type

* Perform the same operations as above, based on school type.

In [39]:
# bins = ["District", "Charter"]
# group_names = ["District", "Charter"]

In [58]:
# school_bins_type = school_summary
# school_bins_type["Type"] = pd.cut(school_summary["Type"], bins, labels=group_names)
# school_bins_type.mean()

# Couldn't get the binning to work using strings, so I just grouped them
school_bins_type = school_summary.groupby("Type").mean()
school_bins_type

,Total Students,Budget,Per Student Budget,Math Score,Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Type,,,,,,,,
Charter,1524.250000,9.126881e+05,599.500000,83.472500,83.897500,93.620000,96.586250,99.21625
District,3853.714286,2.478275e+06,643.571429,76.955714,80.965714,66.548571,80.798571,84.89000
